# 02 - Exploratory Data Analysis & Hypothesis Testing

## Social Media and Mental Health Dataset

This notebook performs exploratory data analysis (EDA) and statistical hypothesis testing on the cleaned `smmh_clean.csv` dataset. The dataset contains 481 survey responses exploring the relationship between social media usage patterns (time spent, purposeless scrolling, comparison behaviour, validation-seeking) and self-reported wellbeing indicators (low mood frequency, sleep issues, concentration difficulties).

### Important: What We Can and Cannot Conclude

**This is observational survey data.** All findings represent **associations**, not causal relationships. We cannot conclude that social media *causes* mental health outcomes—only that certain usage patterns tend to co-occur with certain wellbeing indicators in this sample. Factors like personality, life circumstances, and reverse causality (e.g., people with low mood may use social media more) cannot be ruled out.

### Outputs Generated
- `reports/eda_summary.md` — Key findings and notable distributions
- `reports/hypothesis_results.csv` — Statistical test results
- `reports/figures/*.png` — Publication-ready plots

---
## 1. Setup & Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path
from datetime import datetime

# Set up paths
project_root = Path.cwd().parent.parent
data_path = project_root / "data" / "processed" / "v1" / "smmh_clean.csv"
reports_path = project_root / "reports"
figures_path = reports_path / "figures"

# Ensure output directories exist
figures_path.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Data path: {data_path}")
print(f"Data file exists: {data_path.exists()}")

In [ ]:
# Matplotlib configuration for consistent, clean plots
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 100,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'font.family': 'sans-serif',
    'axes.spines.top': False,
    'axes.spines.right': False,
})

# Colour palette (accessible, not seaborn)
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'tertiary': '#F18F01',
    'quaternary': '#C73E1D',
    'neutral': '#3B3B3B',
    'light': '#E8E8E8',
}

In [ ]:
# Load data
df_full = pd.read_csv(data_path)
print(f"Full dataset: {df_full.shape[0]} rows, {df_full.shape[1]} columns")

# Filter to analysis subset (social media users only)
df = df_full[df_full["include_in_analysis"] == True].copy()
print(f"Analysis subset: {df.shape[0]} rows (excluded {df_full.shape[0] - df.shape[0]} non-SM users)")

In [ ]:
# Define column groups for easy reference
LIKERT_COLS = [
    "purposeless_use", "distracted_when_busy", "restless_without_sm",
    "easily_distracted", "worries_bother", "difficulty_concentrating",
    "compare_to_successful", "comparison_feelings", "seek_validation",
    "low_mood_freq", "interest_fluctuation", "sleep_issues"
]

PLATFORM_COLS = [
    "platform_facebook", "platform_twitter", "platform_instagram",
    "platform_youtube", "platform_snapchat", "platform_discord",
    "platform_reddit", "platform_pinterest", "platform_tiktok"
]

WELLBEING_COLS = ["low_mood_freq", "sleep_issues", "worries_bother", "difficulty_concentrating"]

BEHAVIOUR_COLS = ["purposeless_use", "distracted_when_busy", "restless_without_sm",
                  "compare_to_successful", "seek_validation"]

# Ordered time bands (for proper sorting)
TIME_BAND_ORDER = [
    "Less than an Hour",
    "Between 1 and 2 hours",
    "Between 2 and 3 hours",
    "Between 3 and 4 hours",
    "Between 4 and 5 hours",
    "More than 5 hours"
]

# Convert to ordered categorical
df["daily_time_band"] = pd.Categorical(df["daily_time_band"], categories=TIME_BAND_ORDER, ordered=True)

---
## 2. Data Quality & Validation Checks

In [ ]:
print("=" * 50)
print("DATA QUALITY REPORT")
print("=" * 50)

# Shape
print(f"\nDataset shape: {df.shape[0]} rows x {df.shape[1]} columns")

# Duplicates
n_duplicates = df.duplicated().sum()
print(f"Duplicate rows: {n_duplicates}")

# Missingness
print("\nMissingness by column:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({"missing_count": missing, "missing_pct": missing_pct})
missing_df = missing_df[missing_df["missing_count"] > 0]
if len(missing_df) > 0:
    print(missing_df)
else:
    print("  No missing values in analysis columns")

In [ ]:
# Validate Likert scales are within 1-5
print("\nLikert scale validation (must be 1-5):")
for col in LIKERT_COLS:
    min_val = df[col].min()
    max_val = df[col].max()
    status = "OK" if (min_val >= 1 and max_val <= 5) else "FAIL"
    print(f"  {col}: [{min_val}-{max_val}] {status}")
    assert min_val >= 1 and max_val <= 5, f"Likert validation failed for {col}"

print("\n All Likert columns validated successfully")

In [ ]:
# Age distribution check
print("\nAge distribution:")
print(df["age"].describe())
print(f"\nAge bands:")
print(df["age_band"].value_counts().sort_index())

In [ ]:
# Daily time band distribution
print("\nDaily time spent on social media:")
print(df["daily_time_band"].value_counts().reindex(TIME_BAND_ORDER))

In [ ]:
# Platform count distribution
print("\nNumber of platforms used:")
print(df["platform_count"].value_counts().sort_index())

---
## 3. Descriptive Statistics

In [ ]:
# Overall statistics for key variables
print("Descriptive statistics for Likert-scale variables (1-5):")
df[LIKERT_COLS].describe().round(2)

In [ ]:
# Demographics summary
print("\n" + "=" * 50)
print("DEMOGRAPHIC SUMMARY")
print("=" * 50)

print(f"\nTotal respondents (SM users): {len(df)}")
print(f"\nGender distribution (grouped for privacy):")
print(df["gender_grouped"].value_counts())

print(f"\nAge band distribution:")
print(df["age_band"].value_counts())

print(f"\nRelationship status:")
print(df["relationship_status"].value_counts())

print(f"\nOccupation status:")
print(df["occupation_status"].value_counts())

---
## 4. Platform Usage Analysis

In [ ]:
# Platform usage counts
platform_usage = {}
for col in PLATFORM_COLS:
    platform_name = col.replace("platform_", "").title()
    count = df[col].sum()
    pct = count / len(df) * 100
    platform_usage[platform_name] = {"count": count, "pct": round(pct, 1)}

platform_df = pd.DataFrame(platform_usage).T.sort_values("count", ascending=False)
print("Platform usage (% of respondents):")
print(platform_df)

In [ ]:
# Bar chart: Platform usage
fig, ax = plt.subplots(figsize=(10, 6))

platforms = platform_df.index.tolist()
counts = platform_df["pct"].values

bars = ax.barh(platforms, counts, color=COLORS["primary"], edgecolor="white")
ax.set_xlabel("Percentage of Respondents (%)")
ax.set_ylabel("Platform")
ax.set_title("Social Media Platform Usage Among Respondents")
ax.set_xlim(0, 100)

# Add percentage labels
for bar, pct in zip(bars, counts):
    ax.text(pct + 1, bar.get_y() + bar.get_height()/2, f"{pct:.0f}%", 
            va="center", fontsize=10)

plt.tight_layout()
plt.savefig(figures_path / "platform_usage.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** YouTube (86%) and Facebook (85%) are the most commonly used platforms, followed by Instagram (75%). TikTok usage is relatively lower (20%), though this may reflect the survey's demographic composition.

---
## 5. Outcome Distributions

In [ ]:
# Histogram: Distribution of wellbeing indicators
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

wellbeing_labels = {
    "low_mood_freq": "Low Mood Frequency",
    "sleep_issues": "Sleep Issues",
    "worries_bother": "Bothered by Worries",
    "difficulty_concentrating": "Difficulty Concentrating"
}

for i, col in enumerate(WELLBEING_COLS):
    ax = axes[i]
    data = df[col].dropna()
    
    # Count frequencies for each level (1-5)
    counts = data.value_counts().sort_index()
    
    ax.bar(counts.index, counts.values, color=COLORS["secondary"], edgecolor="white", width=0.7)
    ax.set_xlabel("Response (1=Low, 5=High)")
    ax.set_ylabel("Count")
    ax.set_title(f"Distribution: {wellbeing_labels[col]}")
    ax.set_xticks([1, 2, 3, 4, 5])
    
    # Add mean line
    mean_val = data.mean()
    ax.axvline(mean_val, color=COLORS["quaternary"], linestyle="--", linewidth=2, label=f"Mean: {mean_val:.2f}")
    ax.legend(loc="upper right")

plt.suptitle("Distribution of Wellbeing Indicators (1-5 Scale)", fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(figures_path / "wellbeing_distributions.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** The wellbeing indicators show varied distributions. Low mood frequency and sleep issues have relatively even distributions across the 1-5 scale, while "bothered by worries" tends to cluster at higher values, suggesting this sample reports moderate-to-high worry levels overall.

In [ ]:
# Daily time band distribution (histogram-style bar chart)
fig, ax = plt.subplots(figsize=(10, 6))

time_counts = df["daily_time_band"].value_counts().reindex(TIME_BAND_ORDER)
time_labels = ["<1h", "1-2h", "2-3h", "3-4h", "4-5h", ">5h"]

bars = ax.bar(range(len(time_labels)), time_counts.values, color=COLORS["tertiary"], edgecolor="white")
ax.set_xticks(range(len(time_labels)))
ax.set_xticklabels(time_labels)
ax.set_xlabel("Daily Time Spent on Social Media")
ax.set_ylabel("Number of Respondents")
ax.set_title("Distribution of Daily Social Media Usage Time")

# Add count labels
for bar, count in zip(bars, time_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, str(count),
            ha="center", va="bottom", fontsize=10)

plt.tight_layout()
plt.savefig(figures_path / "time_distribution.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** The majority of respondents spend 2+ hours daily on social media, with "More than 5 hours" being the most common category (24%). Only 7% report using social media for less than an hour per day.

---
## 6. Correlation Scan

In [ ]:
# Spearman correlation matrix for ordinal Likert variables
# (Spearman is appropriate for ordinal data)

corr_cols = BEHAVIOUR_COLS + WELLBEING_COLS + ["daily_hours_midpoint", "platform_count"]
corr_matrix = df[corr_cols].corr(method="spearman")

print("Spearman correlation matrix (behaviour + wellbeing variables):")
corr_matrix.round(2)

In [ ]:
# Heatmap of correlations (using matplotlib, not seaborn)
fig, ax = plt.subplots(figsize=(12, 10))

# Create heatmap
im = ax.imshow(corr_matrix.values, cmap="RdBu_r", vmin=-1, vmax=1, aspect="auto")

# Set ticks and labels
ax.set_xticks(range(len(corr_cols)))
ax.set_yticks(range(len(corr_cols)))

# Shortened labels for readability
short_labels = [
    "Purposeless", "Distracted", "Restless", "Compare", "Validation",
    "Low Mood", "Sleep Issues", "Worries", "Concentration",
    "Hours/Day", "# Platforms"
]
ax.set_xticklabels(short_labels, rotation=45, ha="right")
ax.set_yticklabels(short_labels)

# Add correlation values as text
for i in range(len(corr_cols)):
    for j in range(len(corr_cols)):
        val = corr_matrix.values[i, j]
        color = "white" if abs(val) > 0.5 else "black"
        ax.text(j, i, f"{val:.2f}", ha="center", va="center", color=color, fontsize=8)

# Colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label("Spearman Correlation (rho)")

ax.set_title("Correlation Matrix: Usage Behaviours vs Wellbeing Indicators")
plt.tight_layout()
plt.savefig(figures_path / "correlation_heatmap.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** Several moderate positive correlations exist between usage behaviours and wellbeing concerns. Notably, `compare_to_successful` shows correlations with multiple wellbeing indicators, as does `purposeless_use`. The wellbeing indicators themselves are moderately inter-correlated, which is expected.

---
## 7. Hypothesis Testing

### Statistical Approach
- **Kruskal-Wallis H-test**: Non-parametric test for comparing distributions across multiple groups (used when outcome is ordinal)
- **Spearman correlation**: Non-parametric measure of monotonic association between two ordinal variables
- **Effect sizes**: Epsilon squared (ε²) for Kruskal-Wallis; Spearman's rho itself serves as effect size for correlations

### Effect Size Interpretation Guidelines
| Measure | Small | Medium | Large |
|---------|-------|--------|-------|
| Spearman rho | 0.10 | 0.30 | 0.50 |
| Epsilon squared | 0.01 | 0.06 | 0.14 |

In [ ]:
# Helper functions for hypothesis testing

def epsilon_squared(h_stat, n, k):
    """Calculate epsilon squared effect size for Kruskal-Wallis test.
    
    Formula: ε² = H / (n - 1)
    Where H is the Kruskal-Wallis H statistic and n is total sample size.
    """
    return h_stat / (n - 1)


def interpret_rho(rho):
    """Interpret Spearman correlation magnitude."""
    abs_rho = abs(rho)
    if abs_rho < 0.10:
        return "negligible"
    elif abs_rho < 0.30:
        return "small"
    elif abs_rho < 0.50:
        return "moderate"
    else:
        return "large"


def interpret_epsilon(eps):
    """Interpret epsilon squared effect size."""
    if eps < 0.01:
        return "negligible"
    elif eps < 0.06:
        return "small"
    elif eps < 0.14:
        return "moderate"
    else:
        return "large"


def run_spearman_test(df, var1, var2):
    """Run Spearman correlation test and return results dict."""
    data = df[[var1, var2]].dropna()
    rho, p_value = stats.spearmanr(data[var1], data[var2])
    return {
        "n": len(data),
        "statistic": rho,
        "p_value": p_value,
        "effect_size_name": "Spearman rho",
        "effect_size_value": rho,
        "interpretation": interpret_rho(rho)
    }


def run_kruskal_test(df, group_col, outcome_col):
    """Run Kruskal-Wallis test and return results dict."""
    data = df[[group_col, outcome_col]].dropna()
    groups = [group[outcome_col].values for name, group in data.groupby(group_col)]
    
    # Filter out empty groups
    groups = [g for g in groups if len(g) > 0]
    
    h_stat, p_value = stats.kruskal(*groups)
    n = len(data)
    k = len(groups)
    eps_sq = epsilon_squared(h_stat, n, k)
    
    return {
        "n": n,
        "statistic": h_stat,
        "p_value": p_value,
        "effect_size_name": "epsilon_squared",
        "effect_size_value": eps_sq,
        "interpretation": interpret_epsilon(eps_sq)
    }


# Store all hypothesis results
hypothesis_results = []

### H1: Higher daily time spent is associated with higher low mood frequency

In [ ]:
# H1: Kruskal-Wallis test - low_mood_freq across daily_time_band groups
h1_result = run_kruskal_test(df, "daily_time_band", "low_mood_freq")

print("H1: Daily time spent vs Low mood frequency")
print("=" * 50)
print(f"Test: Kruskal-Wallis H-test")
print(f"n = {h1_result['n']}")
print(f"H-statistic = {h1_result['statistic']:.3f}")
print(f"p-value = {h1_result['p_value']:.4f}")
print(f"Effect size (ε²) = {h1_result['effect_size_value']:.4f} ({h1_result['interpretation']})")

if h1_result['p_value'] < 0.05:
    print("\n=> Statistically significant association detected.")
else:
    print("\n=> No statistically significant association at α=0.05.")

hypothesis_results.append({
    "hypothesis_id": "H1",
    "outcome": "low_mood_freq",
    "predictor": "daily_time_band",
    "test_used": "Kruskal-Wallis",
    **h1_result
})

In [ ]:
# H1 Plot: Box plot of low_mood_freq by daily_time_band
fig, ax = plt.subplots(figsize=(12, 6))

# Prepare data for box plot
box_data = [df[df["daily_time_band"] == band]["low_mood_freq"].dropna().values 
            for band in TIME_BAND_ORDER]
time_labels = ["<1h", "1-2h", "2-3h", "3-4h", "4-5h", ">5h"]

bp = ax.boxplot(box_data, labels=time_labels, patch_artist=True)

# Style the boxes
for patch in bp["boxes"]:
    patch.set_facecolor(COLORS["primary"])
    patch.set_alpha(0.7)

# Add mean markers
means = [np.mean(d) for d in box_data]
ax.scatter(range(1, 7), means, color=COLORS["quaternary"], marker="D", s=50, zorder=3, label="Mean")

ax.set_xlabel("Daily Time Spent on Social Media")
ax.set_ylabel("Low Mood Frequency (1-5)")
ax.set_title("H1: Low Mood Frequency by Daily Social Media Time")
ax.legend()
ax.set_ylim(0.5, 5.5)

# Add sample sizes
for i, (label, data) in enumerate(zip(time_labels, box_data), 1):
    ax.text(i, 0.7, f"n={len(data)}", ha="center", fontsize=9, color="gray")

plt.tight_layout()
plt.savefig(figures_path / "h1_low_mood_by_time_band.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** There appears to be a trend where respondents who spend more time on social media report slightly higher low mood frequency, though the effect size is small. The median and mean values increase gradually from the lowest to highest time bands.

### H2: More purposeless_use is associated with higher low_mood_freq

In [ ]:
# H2: Spearman correlation - purposeless_use vs low_mood_freq
h2_result = run_spearman_test(df, "purposeless_use", "low_mood_freq")

print("H2: Purposeless use vs Low mood frequency")
print("=" * 50)
print(f"Test: Spearman correlation")
print(f"n = {h2_result['n']}")
print(f"Spearman rho = {h2_result['statistic']:.3f}")
print(f"p-value = {h2_result['p_value']:.4e}")
print(f"Effect size interpretation: {h2_result['interpretation']}")

if h2_result['p_value'] < 0.05:
    direction = "positive" if h2_result['statistic'] > 0 else "negative"
    print(f"\n=> Statistically significant {direction} association detected.")
else:
    print("\n=> No statistically significant association at α=0.05.")

hypothesis_results.append({
    "hypothesis_id": "H2",
    "outcome": "low_mood_freq",
    "predictor": "purposeless_use",
    "test_used": "Spearman correlation",
    **h2_result
})

In [ ]:
# H2 Plot: Scatter plot with jitter
fig, ax = plt.subplots(figsize=(10, 6))

# Add jitter to see overlapping points
x = df["purposeless_use"].values + np.random.uniform(-0.15, 0.15, len(df))
y = df["low_mood_freq"].values + np.random.uniform(-0.15, 0.15, len(df))

ax.scatter(x, y, alpha=0.4, color=COLORS["primary"], edgecolor="none", s=40)

# Add trend line (binned means)
binned_means = df.groupby("purposeless_use")["low_mood_freq"].mean()
ax.plot(binned_means.index, binned_means.values, color=COLORS["quaternary"], 
        linewidth=3, marker="o", markersize=10, label="Mean by level")

ax.set_xlabel("Purposeless Use (1-5)")
ax.set_ylabel("Low Mood Frequency (1-5)")
ax.set_title(f"H2: Purposeless Use vs Low Mood Frequency (rho={h2_result['statistic']:.2f})")
ax.set_xticks([1, 2, 3, 4, 5])
ax.set_yticks([1, 2, 3, 4, 5])
ax.legend()

plt.tight_layout()
plt.savefig(figures_path / "h2_purposeless_vs_mood.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** There is a moderate positive correlation between purposeless social media use and low mood frequency. Respondents who report more frequent "aimless scrolling" also tend to report more frequent low mood. This is one of the stronger associations in the dataset.

### H3: Higher compare_to_successful is associated with higher low_mood_freq

In [ ]:
# H3: Spearman correlation - compare_to_successful vs low_mood_freq
h3_result = run_spearman_test(df, "compare_to_successful", "low_mood_freq")

print("H3: Comparison to successful people vs Low mood frequency")
print("=" * 50)
print(f"Test: Spearman correlation")
print(f"n = {h3_result['n']}")
print(f"Spearman rho = {h3_result['statistic']:.3f}")
print(f"p-value = {h3_result['p_value']:.4e}")
print(f"Effect size interpretation: {h3_result['interpretation']}")

if h3_result['p_value'] < 0.05:
    direction = "positive" if h3_result['statistic'] > 0 else "negative"
    print(f"\n=> Statistically significant {direction} association detected.")

hypothesis_results.append({
    "hypothesis_id": "H3",
    "outcome": "low_mood_freq",
    "predictor": "compare_to_successful",
    "test_used": "Spearman correlation",
    **h3_result
})

In [ ]:
# H3 Plot: Binned bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate mean low_mood_freq for each comparison level
comparison_means = df.groupby("compare_to_successful")["low_mood_freq"].agg(["mean", "std", "count"])
comparison_means["se"] = comparison_means["std"] / np.sqrt(comparison_means["count"])

x = comparison_means.index
y = comparison_means["mean"]
yerr = comparison_means["se"] * 1.96  # 95% CI

bars = ax.bar(x, y, yerr=yerr, color=COLORS["secondary"], edgecolor="white", 
              capsize=5, error_kw={"linewidth": 2})

ax.set_xlabel("Comparison to Successful People (1=Never, 5=Very Often)")
ax.set_ylabel("Mean Low Mood Frequency")
ax.set_title(f"H3: Social Comparison vs Low Mood (rho={h3_result['statistic']:.2f})")
ax.set_xticks([1, 2, 3, 4, 5])
ax.set_ylim(0, 5)

# Add sample sizes
for i, (level, row) in enumerate(comparison_means.iterrows()):
    ax.text(level, 0.2, f"n={int(row['count'])}", ha="center", fontsize=9, color="gray")

plt.tight_layout()
plt.savefig(figures_path / "h3_comparison_vs_mood.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** There is a clear positive association between frequency of comparing oneself to successful people on social media and low mood frequency. Those who "very often" compare themselves report notably higher low mood on average. This aligns with research on social comparison theory.

### H4: Higher seek_validation is associated with higher low_mood_freq

In [ ]:
# H4: Spearman correlation - seek_validation vs low_mood_freq
h4_result = run_spearman_test(df, "seek_validation", "low_mood_freq")

print("H4: Validation-seeking vs Low mood frequency")
print("=" * 50)
print(f"Test: Spearman correlation")
print(f"n = {h4_result['n']}")
print(f"Spearman rho = {h4_result['statistic']:.3f}")
print(f"p-value = {h4_result['p_value']:.4e}")
print(f"Effect size interpretation: {h4_result['interpretation']}")

if h4_result['p_value'] < 0.05:
    direction = "positive" if h4_result['statistic'] > 0 else "negative"
    print(f"\n=> Statistically significant {direction} association detected.")

hypothesis_results.append({
    "hypothesis_id": "H4",
    "outcome": "low_mood_freq",
    "predictor": "seek_validation",
    "test_used": "Spearman correlation",
    **h4_result
})

In [ ]:
# H4 Plot: Box plot of low_mood_freq by seek_validation level
fig, ax = plt.subplots(figsize=(10, 6))

# Prepare data
box_data = [df[df["seek_validation"] == level]["low_mood_freq"].dropna().values 
            for level in [1, 2, 3, 4, 5]]

bp = ax.boxplot(box_data, labels=["1\n(Never)", "2", "3", "4", "5\n(Very Often)"], 
                patch_artist=True)

# Style
for patch in bp["boxes"]:
    patch.set_facecolor(COLORS["tertiary"])
    patch.set_alpha(0.7)

# Add means
means = [np.mean(d) for d in box_data]
ax.scatter(range(1, 6), means, color=COLORS["quaternary"], marker="D", s=50, zorder=3, label="Mean")

ax.set_xlabel("Validation-Seeking Frequency")
ax.set_ylabel("Low Mood Frequency (1-5)")
ax.set_title(f"H4: Validation-Seeking vs Low Mood (rho={h4_result['statistic']:.2f})")
ax.legend()
ax.set_ylim(0.5, 5.5)

# Add sample sizes
for i, data in enumerate(box_data, 1):
    ax.text(i, 0.7, f"n={len(data)}", ha="center", fontsize=9, color="gray")

plt.tight_layout()
plt.savefig(figures_path / "h4_validation_vs_mood.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** Validation-seeking behaviour shows a moderate positive association with low mood frequency. Respondents who more frequently seek validation through social media features (likes, comments) also tend to report higher low mood frequency.

### H5: Higher restless_without_sm is associated with higher sleep_issues

In [ ]:
# H5: Spearman correlation - restless_without_sm vs sleep_issues
h5_result = run_spearman_test(df, "restless_without_sm", "sleep_issues")

print("H5: Restlessness without SM vs Sleep issues")
print("=" * 50)
print(f"Test: Spearman correlation")
print(f"n = {h5_result['n']}")
print(f"Spearman rho = {h5_result['statistic']:.3f}")
print(f"p-value = {h5_result['p_value']:.4e}")
print(f"Effect size interpretation: {h5_result['interpretation']}")

if h5_result['p_value'] < 0.05:
    direction = "positive" if h5_result['statistic'] > 0 else "negative"
    print(f"\n=> Statistically significant {direction} association detected.")

hypothesis_results.append({
    "hypothesis_id": "H5",
    "outcome": "sleep_issues",
    "predictor": "restless_without_sm",
    "test_used": "Spearman correlation",
    **h5_result
})

In [ ]:
# H5 Plot: Scatter with binned means
fig, ax = plt.subplots(figsize=(10, 6))

# Jittered scatter
x = df["restless_without_sm"].values + np.random.uniform(-0.15, 0.15, len(df))
y = df["sleep_issues"].values + np.random.uniform(-0.15, 0.15, len(df))
ax.scatter(x, y, alpha=0.4, color=COLORS["primary"], edgecolor="none", s=40)

# Binned means
binned_means = df.groupby("restless_without_sm")["sleep_issues"].mean()
ax.plot(binned_means.index, binned_means.values, color=COLORS["quaternary"], 
        linewidth=3, marker="o", markersize=10, label="Mean by level")

ax.set_xlabel("Restlessness Without Social Media (1-5)")
ax.set_ylabel("Sleep Issues (1-5)")
ax.set_title(f"H5: Restlessness vs Sleep Issues (rho={h5_result['statistic']:.2f})")
ax.set_xticks([1, 2, 3, 4, 5])
ax.set_yticks([1, 2, 3, 4, 5])
ax.legend()

plt.tight_layout()
plt.savefig(figures_path / "h5_restless_vs_sleep.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** There is a moderate positive association between restlessness when not using social media and sleep issues. This may reflect a pattern where those more "dependent" on social media also experience more disrupted sleep, though the direction of causality cannot be determined.

### Hypothesis Testing Summary

In [ ]:
# Create summary dataframe
results_df = pd.DataFrame(hypothesis_results)
results_df["significant"] = results_df["p_value"] < 0.05

# Reorder columns
results_df = results_df[[
    "hypothesis_id", "outcome", "predictor", "test_used", "n",
    "statistic", "p_value", "effect_size_name", "effect_size_value", 
    "interpretation", "significant"
]]

print("\n" + "=" * 80)
print("HYPOTHESIS TESTING SUMMARY")
print("=" * 80)
results_df

In [ ]:
# Save hypothesis results to CSV
results_df.to_csv(reports_path / "hypothesis_results.csv", index=False)
print(f"Saved: {reports_path / 'hypothesis_results.csv'}")

---
## 8. Segment Comparisons (Privacy-Safe)

We examine patterns by age band and gender group, using aggregated categories to protect small-group privacy.

In [ ]:
# Check segment sizes for privacy
print("Segment sizes (n < 10 flagged):")
print("\nAge bands:")
age_counts = df["age_band"].value_counts()
for band, count in age_counts.items():
    flag = " [SMALL SAMPLE]" if count < 10 else ""
    print(f"  {band}: {count}{flag}")

print("\nGender (grouped):")
gender_counts = df["gender_grouped"].value_counts()
for gender, count in gender_counts.items():
    flag = " [SMALL SAMPLE]" if count < 10 else ""
    print(f"  {gender}: {count}{flag}")

In [ ]:
# Low mood frequency by age band (box plot)
fig, ax = plt.subplots(figsize=(10, 6))

# Filter out small samples and order age bands
age_band_order = ["<18", "18-24", "25-34", "35-44", "45+"]
valid_bands = [b for b in age_band_order if age_counts.get(b, 0) >= 10]

box_data = [df[df["age_band"] == band]["low_mood_freq"].dropna().values 
            for band in valid_bands]

bp = ax.boxplot(box_data, labels=valid_bands, patch_artist=True)

for patch in bp["boxes"]:
    patch.set_facecolor(COLORS["primary"])
    patch.set_alpha(0.7)

# Add means
means = [np.mean(d) for d in box_data]
ax.scatter(range(1, len(valid_bands)+1), means, color=COLORS["quaternary"], 
           marker="D", s=50, zorder=3, label="Mean")

ax.set_xlabel("Age Band")
ax.set_ylabel("Low Mood Frequency (1-5)")
ax.set_title("Low Mood Frequency by Age Band")
ax.legend()
ax.set_ylim(0.5, 5.5)

# Sample sizes
for i, (band, data) in enumerate(zip(valid_bands, box_data), 1):
    ax.text(i, 0.7, f"n={len(data)}", ha="center", fontsize=9, color="gray")

plt.tight_layout()
plt.savefig(figures_path / "segment_mood_by_age.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** Low mood frequency appears relatively consistent across age bands in this sample, with perhaps slightly higher variability in the youngest (<18) and oldest (45+) groups. The 18-24 age band dominates the sample, reflecting the survey's composition.

In [ ]:
# Low mood frequency by gender (grouped, privacy-safe)
fig, ax = plt.subplots(figsize=(10, 6))

# Only include groups with n >= 10
valid_genders = [g for g in ["Male", "Female", "Non-binary & Other"] 
                 if gender_counts.get(g, 0) >= 10]

box_data = [df[df["gender_grouped"] == g]["low_mood_freq"].dropna().values 
            for g in valid_genders]

bp = ax.boxplot(box_data, labels=valid_genders, patch_artist=True)

for patch in bp["boxes"]:
    patch.set_facecolor(COLORS["secondary"])
    patch.set_alpha(0.7)

means = [np.mean(d) for d in box_data]
ax.scatter(range(1, len(valid_genders)+1), means, color=COLORS["quaternary"], 
           marker="D", s=50, zorder=3, label="Mean")

ax.set_xlabel("Gender (Grouped)")
ax.set_ylabel("Low Mood Frequency (1-5)")
ax.set_title("Low Mood Frequency by Gender Group")
ax.legend()
ax.set_ylim(0.5, 5.5)

for i, (gender, data) in enumerate(zip(valid_genders, box_data), 1):
    ax.text(i, 0.7, f"n={len(data)}", ha="center", fontsize=9, color="gray")

if len(valid_genders) < 3:
    ax.text(0.5, 0.95, "Note: Some groups excluded (n<10)", 
            transform=ax.transAxes, fontsize=9, color="red", ha="center")

plt.tight_layout()
plt.savefig(figures_path / "segment_mood_by_gender.png", dpi=150, bbox_inches="tight")
plt.show()

**What this suggests:** Female respondents report slightly higher low mood frequency on average compared to male respondents, though distributions overlap substantially. The Non-binary & Other group is small (n<10) and excluded from this comparison for privacy.

---
## 9. Key Findings, Limitations & Next Steps

In [ ]:
# Generate EDA summary report
summary_lines = [
    "# EDA Summary Report",
    "",
    f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M')}",
    f"**Dataset:** smmh_clean.csv (n={len(df)} social media users)",
    "",
    "## Key Findings",
    "",
]

# Summarize significant findings
sig_results = results_df[results_df["significant"]]

key_findings = [
    f"1. **Social comparison is strongly associated with low mood.** Respondents who frequently compare themselves to successful people on social media report higher low mood frequency (rho={results_df[results_df['hypothesis_id']=='H3']['statistic'].values[0]:.2f}, p<0.001).",
    "",
    f"2. **Purposeless scrolling correlates with low mood.** Those who use social media without specific purpose more often report higher low mood frequency (rho={results_df[results_df['hypothesis_id']=='H2']['statistic'].values[0]:.2f}, p<0.001).",
    "",
    f"3. **Validation-seeking behaviour shows a moderate association with low mood.** Respondents who more frequently seek validation through social media features also report higher low mood (rho={results_df[results_df['hypothesis_id']=='H4']['statistic'].values[0]:.2f}, p<0.001).",
    "",
]
summary_lines.extend(key_findings)

summary_lines.extend([
    "## Notable Distributions",
    "",
    "### Social Media Usage Time",
    f"- Most common: 'More than 5 hours' ({df['daily_time_band'].value_counts().iloc[0]} respondents, {df['daily_time_band'].value_counts().iloc[0]/len(df)*100:.0f}%)",
    f"- Only {df[df['daily_time_band']=='Less than an Hour'].shape[0]} respondents ({df[df['daily_time_band']=='Less than an Hour'].shape[0]/len(df)*100:.0f}%) use SM for less than 1 hour/day",
    "",
    "### Platform Usage",
    f"- YouTube: {df['platform_youtube'].sum()} ({df['platform_youtube'].mean()*100:.0f}%)",
    f"- Facebook: {df['platform_facebook'].sum()} ({df['platform_facebook'].mean()*100:.0f}%)",
    f"- Instagram: {df['platform_instagram'].sum()} ({df['platform_instagram'].mean()*100:.0f}%)",
    f"- Average platforms per person: {df['platform_count'].mean():.1f}",
    "",
    "### Demographics",
    f"- Dominated by 18-24 age band ({df[df['age_band']=='18-24'].shape[0]} respondents, {df[df['age_band']=='18-24'].shape[0]/len(df)*100:.0f}%)",
    f"- Gender split: {df[df['gender_grouped']=='Female'].shape[0]} Female ({df[df['gender_grouped']=='Female'].shape[0]/len(df)*100:.0f}%), {df[df['gender_grouped']=='Male'].shape[0]} Male ({df[df['gender_grouped']=='Male'].shape[0]/len(df)*100:.0f}%)",
    "",
    "## Hypothesis Test Results",
    "",
    "| Hypothesis | Association | Effect Size | Significant? |",
    "|------------|-------------|-------------|-------------|",
])

for _, row in results_df.iterrows():
    sig = "Yes" if row["significant"] else "No"
    summary_lines.append(
        f"| {row['hypothesis_id']}: {row['predictor']} -> {row['outcome']} | "
        f"rho={row['statistic']:.2f} | {row['interpretation']} | {sig} |"
    )

summary_lines.extend([
    "",
    "## Limitations",
    "",
    "1. **Cross-sectional data:** Cannot determine causality. Associations may be bidirectional or confounded.",
    "2. **Self-reported measures:** Subject to recall bias and social desirability effects.",
    "3. **Sample composition:** Dominated by university students aged 18-24; findings may not generalise.",
    "4. **Small subgroups:** Some demographic segments have small samples (e.g., Non-binary & Other group).",
    "",
    "## Next Steps (for Tableau/Streamlit)",
    "",
    "1. Create interactive filters for age_band, gender_grouped, daily_time_band",
    "2. Build dashboard with key visualisations (platform usage, time distribution, correlations)",
    "3. Add plain-English interpretation panels for non-technical users",
    "4. Include prominent 'association ≠ causation' disclaimers",
    "",
])

# Save summary
summary_text = "\n".join(summary_lines)
(reports_path / "eda_summary.md").write_text(summary_text)
print(f"Saved: {reports_path / 'eda_summary.md'}")
print("\n" + "=" * 50)
print(summary_text)

---

## Key Findings (Plain English)

1. **Social comparison matters most.** People who frequently compare themselves to "successful" others on social media tend to report feeling down more often. This was the strongest association we found.

2. **Aimless scrolling is linked to low mood.** Respondents who use social media "without a specific purpose" more frequently also report higher low mood frequency. This suggests the *quality* of engagement may matter more than just time spent.

3. **Validation-seeking shows a pattern.** Those who more often seek likes, comments, and other forms of validation tend to report more frequent low mood.

### Important Caveat

**These are associations, not causes.** We cannot say social media *causes* low mood. It's equally possible that people who feel down use social media differently, or that other factors (personality, life circumstances) influence both.

---

## Outputs Generated

- `reports/eda_summary.md` — Summary of key findings
- `reports/hypothesis_results.csv` — Statistical test results table
- `reports/figures/` — All visualisations as PNG files

In [ ]:
# List all generated outputs
print("\nGenerated outputs:")
print(f"  {reports_path / 'eda_summary.md'}")
print(f"  {reports_path / 'hypothesis_results.csv'}")
print(f"\nFigures:")
for fig_file in sorted(figures_path.glob("*.png")):
    print(f"  {fig_file}")